In [2]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
from decimal import Decimal
import hashlib

# RPC Configuration
rpc_user = "nidarsana02"
rpc_password = "hjsHjZR1OIAft7Y-lYScC9F94PR54eAvp0vGPurIY1w"
rpc_host = "127.0.0.1"
rpc_port = 18443
wallet_name = "CS216"

# Connect to Bitcoin RPC
rpc_client = AuthServiceProxy(f"http://{rpc_user}:{rpc_password}@{rpc_host}:{rpc_port}/wallet/{wallet_name}")

In [3]:
# Ensure wallet is loaded
def ensure_wallet_loaded(wallet_name):
    try:
        loaded_wallets = rpc_client.listwallets()
        if wallet_name in loaded_wallets:
            print(f"Wallet '{wallet_name}' is already loaded.")
            return
        print(f"Loading wallet '{wallet_name}'...")
        rpc_client.loadwallet(wallet_name)
    except JSONRPCException as e:
        if e.error.get("code") == -18:  # Wallet does not exist
            print(f"Wallet '{wallet_name}' not found. Creating...")
            rpc_client.createwallet(wallet_name)
        else:
            raise

ensure_wallet_loaded(wallet_name)

# Generate P2SH-SegWit Addresses
address_A = rpc_client.getnewaddress("A'", "p2sh-segwit")
address_B = rpc_client.getnewaddress("B'", "p2sh-segwit")
address_C = rpc_client.getnewaddress("C'", "p2sh-segwit")

print(f"Address A: {address_A}")
print(f"Address B: {address_B}")
print(f"Address C: {address_C}")

Wallet 'CS216' is already loaded.
Address A: 2N8Z1EwkA1jMy94qduycaXi118PiuGGsQLQ
Address B: 2N9jBHLYmuxoXVPNhMoX9B6nCzhWtgLR5fe
Address C: 2MwhJMmd9aLQuTrVaLYeHjobRBm3zggHjs3


In [4]:
# Fund Address A
rpc_client.sendtoaddress(address_A, 0.01)
rpc_client.generatetoaddress(1, address_A)  # Confirm transaction by mining 1 block

# Get UTXO for Address A
unspent_outputs = rpc_client.listunspent(0, 9999999, [address_A])
if not unspent_outputs:
    raise ValueError("No UTXOs found for Address A.")

utxo = unspent_outputs[0]
amount = Decimal(str(utxo["amount"]))

# Create Transaction A -> B
raw_tx = rpc_client.createrawtransaction(
    [{"txid": utxo["txid"], "vout": utxo["vout"]}],
    {address_B: amount - Decimal("0.0001")}
)

In [5]:
signed_tx = rpc_client.signrawtransactionwithwallet(raw_tx)
decoded_tx = rpc_client.decoderawtransaction(signed_tx["hex"])
print("Decoded Transaction A -> B:", decoded_tx)
txid = rpc_client.sendrawtransaction(signed_tx["hex"])
print("Transaction ID A -> B:", txid)

# Get UTXO for Address B
unspent_outputs_B = rpc_client.listunspent(0, 9999999, [address_B])
if not unspent_outputs_B:
    raise ValueError("No UTXOs found for Address B.")

utxo_B = unspent_outputs_B[0]
amount_B = Decimal(str(utxo_B["amount"]))

Decoded Transaction A -> B: {'txid': 'e24b55e6d4b9a323ca1b84894e4278bf9e8ed1f541125aba4a10b1bdb419dce0', 'hash': '24eb46d849ab4103cb04191e5842bb2ea27ad5cfcb4f0cfba2889a25e0e9a2bc', 'version': 2, 'size': 215, 'vsize': 134, 'weight': 533, 'locktime': 0, 'vin': [{'txid': '0f062d0ced04d25c2c9d4d726027d1e6313020d5ab2cb72a69263d9557bfb286', 'vout': 0, 'scriptSig': {'asm': '00143e2000fc3f63e88972392d769b6b1b321d169865', 'hex': '1600143e2000fc3f63e88972392d769b6b1b321d169865'}, 'txinwitness': ['3044022068183d30e406d7115c6d76bd05bb2dcc44d3c83c3876291610c384fd521739170220552eb211771fdee49b44607a631dd1abe2cac11b702f85cff33abfbce809045501', '034c032633b9ce1f3f554e0f44da4f7be638182fb7d829d9cdd74fe3ee87ba378e'], 'sequence': 4294967293}], 'vout': [{'value': Decimal('0.00990000'), 'n': 0, 'scriptPubKey': {'asm': 'OP_HASH160 b4cb6690868781ea6bd0e36ef42ae205ae458f1d OP_EQUAL', 'desc': 'addr(2N9jBHLYmuxoXVPNhMoX9B6nCzhWtgLR5fe)#acrkg4lh', 'hex': 'a914b4cb6690868781ea6bd0e36ef42ae205ae458f1d87', 'address'

In [6]:
# Create Transaction B -> C
raw_tx_B = rpc_client.createrawtransaction(
    [{"txid": utxo_B["txid"], "vout": utxo_B["vout"]}],
    {address_C: amount_B - Decimal("0.0001")}
)

signed_tx_B = rpc_client.signrawtransactionwithwallet(raw_tx_B)
decoded_tx_B = rpc_client.decoderawtransaction(signed_tx_B["hex"])
print("Decoded Transaction B -> C:", decoded_tx_B)
txid_B = rpc_client.sendrawtransaction(signed_tx_B["hex"])
print("Transaction ID B -> C:", txid_B)

Decoded Transaction B -> C: {'txid': 'f2bdd1449ae7f8c4ecc91c4a3afb3f53f776abde8703606dec37242699c5f60a', 'hash': '5ff30609d496aa8dcf6319818966fb51e1c44c80c9b131988bd46bab0772f7e5', 'version': 2, 'size': 215, 'vsize': 134, 'weight': 533, 'locktime': 0, 'vin': [{'txid': 'e24b55e6d4b9a323ca1b84894e4278bf9e8ed1f541125aba4a10b1bdb419dce0', 'vout': 0, 'scriptSig': {'asm': '0014345d301aed4f75d42447639d275ae80279f8a0f5', 'hex': '160014345d301aed4f75d42447639d275ae80279f8a0f5'}, 'txinwitness': ['304402204b324a3740e72cb8c015a2b07fac722312b10fd18b948c97ebc97c2ca98a35a40220411769bfb7368a03d0252e596c2a32837b26494284b5765a0a3be5c6bbbb9cef01', '0379f1b04400212166c1de756142ddafcec54e620b9e5f48fb4d2624e4d0d9e5f0'], 'sequence': 4294967293}], 'vout': [{'value': Decimal('0.00980000'), 'n': 0, 'scriptPubKey': {'asm': 'OP_HASH160 30cef766bd8ddd8da28ef91a3744b48b437ccfb0 OP_EQUAL', 'desc': 'addr(2MwhJMmd9aLQuTrVaLYeHjobRBm3zggHjs3)#a9lkj277', 'hex': 'a91430cef766bd8ddd8da28ef91a3744b48b437ccfb087', 'address'

In [7]:
# Extract Public Key from Transaction B (txinwitness[1])
public_key = decoded_tx_B["vin"][0]["txinwitness"][1]  # Second witness (Public Key)

# Compute HASH160(pubKey)
pubkey_hash = hashlib.new("ripemd160", hashlib.sha256(bytes.fromhex(public_key)).digest()).hexdigest()

# Create redeem script for P2SH-P2WPKH (0014 + pubkey_hash)
redeem_script = "0014" + pubkey_hash  # Ensure correct formatting

# Compute HASH160(redeem_script)
redeem_script_hash = hashlib.new("ripemd160", hashlib.sha256(bytes.fromhex(redeem_script)).digest()).hexdigest()

# Extract Expected Redeem Script Hash from scriptPubKey
expected_redeem_script_hash = decoded_tx["vout"][0]["scriptPubKey"]["asm"].split(" ")[1]  # Extract the hash from OP_HASH160

# Check if the computed hash matches the expected hash
if redeem_script_hash == expected_redeem_script_hash:
    print("✅ Hash matches! Transaction is valid.")
else:
    print("❌ Hash mismatch! Check public key and script format.")

# Debugging Info
print(f"Public Key: {public_key}")
print(f"Computed Redeem Script Hash: {redeem_script_hash}")
print(f"Expected Redeem Script Hash: {expected_redeem_script_hash}")


✅ Hash matches! Transaction is valid.
Public Key: 0379f1b04400212166c1de756142ddafcec54e620b9e5f48fb4d2624e4d0d9e5f0
Computed Redeem Script Hash: b4cb6690868781ea6bd0e36ef42ae205ae458f1d
Expected Redeem Script Hash: b4cb6690868781ea6bd0e36ef42ae205ae458f1d
